In [1]:
from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-2.4.2         |   py36h1ba5d50_0         618 KB
    openssl-1.1.1a             |    h14c3975_1000         4.0 MB  conda-forge
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    conda-4.6.2                |           py36_0         869 KB  conda-forge
    libssh2-1.8.0              |                1         239 KB  conda-forge
    python-3.6.8               |       h0371630_0        34.4 MB
    ------------------------------------------------------------
                                           Total:        42.6 MB

The following packages will be UPDATED:

    conda:        4.5.12-py36_1000         conda

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [3]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [4]:
# loop through to find postcode, borough, neighborhood 
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df.shape

(103, 3)

In [6]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [7]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [8]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
len(df2['Borough'].unique())

11

In [10]:
df2.shape

(103, 3)

In [11]:
# !pip install geocoder
"""
Collecting geocoder
  Downloading https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl (98kB)
Requirement already satisfied: click in c:\programdata\anaconda3\lib\site-packages (from geocoder) (6.7)
Requirement already satisfied: six in c:\programdata\anaconda3\lib\site-packages (from geocoder) (1.11.0)
Collecting ratelim (from geocoder)
  Downloading https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl
Requirement already satisfied: requests in c:\programdata\anaconda3\lib\site-packages (from geocoder) (2.19.1)
Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from geocoder) (0.16.0)
Requirement already satisfied: decorator in c:\programdata\anaconda3\lib\site-packages (from ratelim->geocoder) (4.3.0)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (3.0.4)
Requirement already satisfied: idna<2.8,>=2.5 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (2.7)
Requirement already satisfied: urllib3<1.24,>=1.21.1 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (1.23)
Requirement already satisfied: certifi>=2017.4.17 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (2018.8.24)
Installing collected packages: ratelim, geocoder
Successfully installed geocoder-1.38.1 ratelim-0.1.6
"""

print('geocoder has been installed before.')
!conda install -c conda-forge geocoder --yes
import geocoder
print('geocoder has been successfully imported.')

geocoder has been installed before.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_0
  orderedset         conda-forge/linux-64::orderedset-2.0-py36_0
  ratelim            conda-forge/linux-64::ratelim-0.1.6-py36_0



orderedset-2.0       | 231 KB    | ##################################### | 10

In [12]:
address = 'Toronto,Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_x, longitude_y))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
import numpy as np
import pandas as pd
df3 = pd.read_csv('data.csv')

In [14]:
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70976500000006, -79.36379132299999]

In [16]:
postal_codes = df3['Postal Code']    
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [26]:
postal_codes

0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
5      M1J
6      M1K
7      M1L
8      M1M
9      M1N
10     M1P
11     M1R
12     M1S
13     M1T
14     M1V
15     M1W
16     M1X
17     M2H
18     M2J
19     M2K
20     M2L
21     M2M
22     M2N
23     M2P
24     M2R
25     M3A
26     M3B
27     M3C
28     M3H
29     M3J
30     M3K
31     M3L
32     M3M
33     M3N
34     M4A
35     M4B
36     M4C
37     M4E
38     M4G
39     M4H
40     M4J
41     M4K
42     M4L
43     M4M
44     M4N
45     M4P
46     M4R
47     M4S
48     M4T
49     M4V
50     M4W
51     M4X
52     M4Y
53     M5A
54     M5B
55     M5C
56     M5E
57     M5G
58     M5H
59     M5J
60     M5K
61     M5L
62     M5M
63     M5N
64     M5P
65     M5R
66     M5S
67     M5T
68     M5V
69     M5W
70     M5X
71     M6A
72     M6B
73     M6C
74     M6E
75     M6G
76     M6H
77     M6J
78     M6K
79     M6L
80     M6M
81     M6N
82     M6P
83     M6R
84     M6S
85     M7A
86     M7R
87     M7Y
88     M8V
89     M8W
90     M8X

In [17]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df3['Latitude'] = df_coords['Latitude']
df3['Longitude'] = df_coords['Longitude']
df3['Longitude']

0     -79.195561
1     -79.158701
2     -79.175299
3     -79.217610
4     -79.239440
5     -79.232172
6     -79.263670
7     -79.284910
8     -79.234976
9     -79.260069
10    -79.268974
11    -79.300560
12    -79.267976
13    -79.299244
14    -79.280147
15    -79.320740
16    -79.216701
17    -79.356236
18    -79.347782
19    -79.380542
20    -79.379940
21    -79.413546
22    -79.407420
23    -79.400062
24    -79.445700
25    -79.329268
26    -79.362212
27    -79.343415
28    -79.448621
29    -79.488502
30    -79.467320
31    -79.505004
32    -79.496750
33    -79.519590
34    -79.313340
35    -79.311803
36    -79.307165
37    -79.295410
38    -79.363791
39    -79.349881
40    -79.334175
41    -79.355120
42    -79.314642
43    -79.334830
44    -79.387090
45    -79.388514
46    -79.406960
47    -79.385769
48    -79.382973
49    -79.402335
50    -79.378445
51    -79.366600
52    -79.381302
53    -79.362640
54    -79.378180
55    -79.375481
56    -79.373675
57    -79.384930
58    -79.3825

In [20]:
df3['Borough']=df2['Borough']
df3['Neighborhood']=df2['Neighborhood']

In [22]:
df3.shape

(103, 5)

In [24]:
df3.head(10)

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.811650,-79.195561,Scarborough,"Rouge, Malvern"
1,M1C,43.785605,-79.158701,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.765690,-79.175299,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.768216,-79.217610,Scarborough,Woburn
4,M1H,43.769608,-79.239440,Scarborough,Cedarbrae
5,M1J,43.743085,-79.232172,Scarborough,Scarborough Village
6,M1K,43.726260,-79.263670,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,43.713213,-79.284910,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,43.723575,-79.234976,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,43.696690,-79.260069,Scarborough,"Birch Cliff, Cliffside West"
